<a href="https://www.kaggle.com/code/albertomonedero/fashioncnn?scriptVersionId=132148689" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# SISTEMA DE RECOMENDACIÓN DE ARTÍCULOS DE MODA

In [ ]:
from numpy import loadtxt
import pandas as pd
from PIL import Image
from fastcore.all import *
from torch.utils.data import Dataset ,DataLoader
import torchvision.transforms as transforms
from tqdm.notebook import tqdm
import torch
import numpy
from torch import nn
import torchmetrics as metrics
import pytorch_lightning as pl
import matplotlib.pyplot as plt 


dataFrame = pd.read_csv('../input/fashion-product-images-small/styles.csv',on_bad_lines='skip').dropna()
dataFrame


In [ ]:
import sys
print(sys.version)
print(numpy.__version__)

# **Preparación de los datos de entrada (CSV)**
Como quiero clasificar las prendas únicamente por género y estilo, creo un nuevo csv a mi gusto, quedándome sólo con dos columnas:
* image = nombre del archivo de imagen, ej ->"15970.jpg".
* target = columna objetivo,representada por un entero que equivale a un usage+gender concreto
            ej -> "0", que sería CasualMen.

Además debo eliminar ciertas entradas del csv original erróneas, ya que existen algunas entradas con ids que no se corresponden a ninguna imágen de la carpeta de imágenes.


In [ ]:
#creo la columna image con id+.jpg y la columna target con usage+gender

def getImageName(id): return '{name}.jpg'.format(name = id)

dataFrame['image'] = dataFrame['id'].map(getImageName)
dataFrame['target'] =dataFrame['usage']+dataFrame['gender']

dataFrame.head()

In [ ]:
#nos quedamos solo con las columnas image y target
ds = dataFrame.filter(['image', 'target'])
ds.head()


In [ ]:
#Tenemos 25 clases objetivo en nuestro dataframe
targets=ds.target.unique()
CLASSES=len(targets)
print(CLASSES)
print(targets)

In [ ]:
#Creo un diccionario para mapear targets con enteros
#Sustituyo en el dataFrame el nombre de los targets por el número correspondiente en dicho diccionario
i=0
classDic={}
for t in targets:
    ds['target']=ds['target'].replace(t,i)
    classDic[i]=t
    i+=1
print(ds)
print(classDic)

In [ ]:

#Procedo a eliminar las filas con ids de imágenes que no existen en la carpeta de imágenes
images=ds.image.unique()
imagenes_existentes=os.listdir('../input/fashion-product-images-small/images')
for i in images:
    if(i not in imagenes_existentes):
        ds=ds.drop(ds[ds['image']==i].index)

In [ ]:
#guardo mi nuevo dataframe en un csv
ds.to_csv('custom.csv',index=False)


# Creación del dataset personalizado

In [ ]:
transformaciones = transforms.Compose([transforms.Resize((64,64)),
                                      transforms.ToTensor(), #0 - 255--->0 - 1
                                      transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]) #0 - 1--->-1 - 1
                                     ])
batch_size = 64

In [ ]:

class CustomDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform):
        self.annotations = pd.read_csv(csv_file, on_bad_lines='skip')
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        img_path = os.path.join(self.root_dir, self.annotations.iloc[index, 0])
        image = Image.open(img_path).convert('RGB')
        y_label = torch.tensor(int(self.annotations.iloc[index, 1]))
        image = self.transform(image)

        return (image, y_label)


#Instanciamos el dataset con nuestro csv personalizado, las imágenes 
#y las transformaciones que se le aplican a las mismas 
dataset = CustomDataset(
    csv_file="./custom.csv",
    root_dir="../input/fashion-product-images-small/images/",
    transform=transformaciones,
)
print(len(dataset))

In [ ]:
torch.cuda.is_available()

In [ ]:
#Dividimos el dataset en 3:entrenamiento, validación y test, y creamos los dataloaders
train_set,val_set, test_set = torch.utils.data.random_split(dataset, [30000, 9072,5000])
train_loader = DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True, pin_memory=True, num_workers=2)
test_loader = DataLoader(dataset=test_set, batch_size=batch_size, num_workers=2)
val_loader = DataLoader(dataset=val_set, batch_size=batch_size,  num_workers=2)

# Primera Fase: CNN para clasificar imágenes por género y estilo.


# Creación del Modelo CNN

In [ ]:
class CNNModel(pl.LightningModule):
    def __init__(self):
        #shape=[batchsize,canalesentrada,alto,ancho]
        #entrada [64, 3, 64, 64])
        super().__init__()
    #EXTRACCIÓN DE CARACTERÍSTICAS
        #1 BLOQUE CONV
        self.cnv = nn.Conv2d(3,40,5,2)#[64, 40, 30, 30]
        self.rel = nn.ReLU()                    
        self.bn = nn.BatchNorm2d(40)  
        self.mxpool = nn.MaxPool2d(2)#[64, 40, 15, 15]
        #2 BLOQUE CONV
        self.cnv2 = nn.Conv2d(40,55,5,2)#[64, 55, 6, 6])
        self.rel2 = nn.ReLU()                    
        self.bn2 = nn.BatchNorm2d(55)  
        self.mxpool2 = nn.MaxPool2d(2)#[64, 55, 3, 3]
        
        
    #CARACTERIZACIÓN
        self.flat = nn.Flatten()#[64,55x3x3=495]
        self.fc1 = nn.Linear(495,495)
        self.fc2 = nn.Linear(495,300)
        self.fc3 = nn.Linear(300,CLASSES)
        self.softmax = nn.Softmax()
        self.accuracy = metrics.Accuracy(task='multiclass',num_classes=25) #predicciones correctas/total de predicciones

    def forward(self,x):
       # print('antes conv',x.shape)
        out = self.cnv(x)
        #print('despues conv',out.shape)
        out = self.rel(out)
       # print('despues relu',out.shape)
        out = self.bn(out)
        #print('despues batchnorm',out.shape)
        out = self.mxpool(out)
        #rint('despues maxpool',out.shape)
        
        
        out = self.cnv2(out)
        #print('despues conv2',out.shape)
        out = self.rel2(out)
        #print('despues relu2',out.shape)
        out = self.bn2(out)
        #print('despues batchnorm2',out.shape)
        out = self.mxpool2(out)
       # print('despues maxpool2',out.shape)
        
        out = self.flat(out)
       # print('despues flat',out.shape)
        out = self.rel(self.fc1(out))
        out = self.rel(self.fc2(out))   
        out = self.fc3(out)
        return out

    def loss_fn(self,out,target):
        return nn.CrossEntropyLoss()(out.view(-1,CLASSES),target)
    
    def configure_optimizers(self):
        LR = 1e-3
        optimizer = torch.optim.AdamW(self.parameters(),lr=LR)
        return optimizer
    
    def predict(self, x):
        with torch.no_grad():
            out = self(x)
            out=nn.Softmax(-1)(out)
            return torch.argmax(out, axis=1)

    def training_step(self,batch,batch_idx):
        x,y = batch
        imgs = x.view(-1,3,64,64)
        labels = y.view(-1)
        out = self(imgs)
        loss = self.loss_fn(out,labels)
        out = nn.Softmax(-1)(out)
        logits = torch.argmax(out,dim=1)
        accu = self.accuracy(logits, labels)
        self.log('train_loss', loss, prog_bar=True)
        self.log('train_acc', accu, prog_bar=True)
        return loss       

    def validation_step(self,batch,batch_idx):
        x,y = batch
        imgs = x.view(-1,3,64,64)
        labels = y.view(-1)
        out = self(imgs)
        loss = self.loss_fn(out,labels)
        out = nn.Softmax(-1)(out) 
        logits = torch.argmax(out,dim=1)
        accu = self.accuracy(logits, labels)
        self.log('val_loss', loss, prog_bar=True)
        self.log('val_acc', accu, prog_bar=True)
        return loss, accu

In [ ]:
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
print("Device:", device)

In [ ]:
mod = CNNModel().to(device)


Exportamos el modelo en formato ONNX para hacer uso de la aplicación Netron, que permite visualizar la arquitectura de un modelo de deep learning.


In [ ]:
import torch.onnx

input_names = ["input"]
output_names = ["output"]



batch=next(iter(test_loader)) #cogemos 1 lote, batch=[[imgs],[labels]]
imgs, labels= batch[0],batch[1]



# Exportamos el modelo a ONNX
torch.onnx.export(mod, imgs.to(device), "model.onnx", input_names=input_names, output_names=output_names, opset_version=11)


# Entrenamiento del modelo


In [ ]:
mod.train()
trainer = pl.Trainer(accelerator='gpu',devices=1,
                     max_epochs=4
                    
)
trainer.fit(mod,train_loader,val_loader) 

In [ ]:
#Guardamos el estado del modelo entrenado
state_dict = mod.state_dict()
torch.save(state_dict, "prueba7.pth")

# Pruebas: Calcular el rendimiento con datos de Test, obtener matriz de confusión y probar a predecir una imágen


In [ ]:
#Cargo mi modelo ya entrenado
state = torch.load("../input/modelo6/prueba6.pth") #por ahora modelo 6 es el mejor

new_model =CNNModel()
new_model.load_state_dict(state)

In [ ]:
print(len(test_set),len(test_loader))

Cálculo de rendimiento

In [ ]:
    #Calculamos la acc y loss del modelo 
    with torch.no_grad():
        correct = 0
        total = 0
        total_loss=0
        new_model.eval()
        for data, target in test_loader: #vamos iterando los distintos lotes del test_loader
            images = data
            labels =target
            outputs = new_model(images)
            total_loss+=new_model.loss_fn(outputs,labels)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
                

            accuracy = correct / total

        print('Test Accuracy of the model: {} %'.format(100 * correct / total))
        print('loss',total_loss/len(test_loader))  #divido la perdida en cada lote por el numero de lotes
 


In [ ]:
print(targets)
print(classDic)

Matriz de confusión

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sn


# Crear listas de etiquetas verdaderas y predichas
all_labels = list(range(len(targets)))
predicted_labels = []
true_labels = []
for data, target in test_loader:
    images = data
    labels = target
    outputs = new_model(images)
    _, predicted = torch.max(outputs.data, 1)
    predicted_labels.extend(predicted.tolist())
    true_labels.extend(labels.tolist())

# Agregar las etiquetas faltantes a las listas correspondientes
missing_labels = list(set(all_labels) - set(predicted_labels))
predicted_labels += missing_labels
true_labels += missing_labels

# Calcular la matriz de confusión
cm = confusion_matrix(true_labels, predicted_labels)

# Imprimir la matriz de confusión
df_cm = pd.DataFrame(cm, index=targets, columns=targets)
plt.figure(figsize=(30, 25))
sn.set(font_scale=1.8)
sn.heatmap(df_cm, annot=True, cmap="Blues", fmt='g', linewidths=1.5, square=True)
plt.xlabel('Predicho')
plt.ylabel('Verdadero')
plt.title('Matriz de confusión', fontsize=18)
plt.show()



In [ ]:
#Obtenemos las metricas de la matriz de confusion
from sklearn.metrics import classification_report

print(classification_report(true_labels, predicted_labels, target_names=targets))

ejemplo de predicción

In [ ]:
#PROBAMOS A PREDECIR 1 DE LAS IMAGENES DEL TEST_LOADER


batch=next(iter(test_loader)) #cogemos 1 lote, batch=[[imgs],[labels]]
imgs, labels= batch[0],batch[1]
im=imgs[1]
label=labels[1]
plt.imshow(im.numpy()[0], cmap='gray')
print("valor esperado:",classDic[label.item()])


In [ ]:
output = new_model(im.unsqueeze(0))
_, predicted = torch.max(output, 1)
etiqueta =classDic[predicted.item()]
print('Valor predicho: ',etiqueta )

In [ ]:
#lo mismo pero probando la función predict del modelo
prediction = new_model.predict(im.unsqueeze(0))

etiqueta2=classDic[prediction.item()]
print('Valor predicho: ',etiqueta2 )

# Segunda Fase: Tras clasificar una prenda, obtener los candidatos a recomendar (aquellos con la misma etiqueta) y devolver los N candidatos más parecidos.


Clasifico una prenda

In [ ]:
#imagen a clasificar
img = Image.open('../input/fashion-product-images-small/images/10080.jpg')
img

In [ ]:
#clasificar dicha imagen
t_img = transformaciones(img).unsqueeze(0)
output = new_model(t_img)
_, predicted = torch.max(output, 1)
etiqueta =classDic[predicted.item()]
print('Valor predicho: ',etiqueta )


In [ ]:
#lo mismo pero usando la función prediction definida en el modelo
t_img = transformaciones(img).unsqueeze(0)
prediction = new_model.predict(t_img)

etiqueta2=classDic[prediction.item()]
print('Valor predicho: ',etiqueta2 )

A continuación, devuelvo los posibles candidatos ( en este caso, prendas deportivas de hombre)

In [ ]:
print(ds)
print(classDic)

In [ ]:
#Creo una condición para obtener candidatas del dataframe
condicion_etiqueta = ds['target'] == predicted.item()
condicion_etiqueta.head()
            

In [ ]:
candidatas=ds[condicion_etiqueta]
print(candidatas)

# Función para devolver las N candidatas más parecidas a una imagen dada

Uso la librería img2vect de pytorch que usa modelos pre-entrenados (en mi caso resnet-18) para obtener la representación vectorial de las imágenes, y después calcular la similitud usando como función la similitud coseno.

In [ ]:
pip install img2vec-pytorch

In [ ]:
from img2vec_pytorch import Img2Vec
img2vec = Img2Vec(cuda=True, model='resnet-18')
cos = nn.CosineSimilarity(eps=1e-6)
imagenes=candidatas.image.unique()

esta función no devuelve las recomendaciones ordenadas directamente, habría que ordenarlas después

In [ ]:

def N_mas_parecidas(imagen,n,imagenes):
   
    vec1 = img2vec.get_vec(imagen, tensor=True).reshape(512) #representación vectorial imagen de entrada
    dic={}#diccionario con imagenes y similitudes
    similitudes=[]
    recomendadas={}
    for im in imagenes: #cargo el diccionario con cada imagen como clave y la similitud como valor
                        #y obtengo una lista con las n mayores similitudes, ordenada de mayor a menor
        candidata = Image.open('../input/fashion-product-images-small/images/'+im)       
        vec2 = img2vec.get_vec(candidata.convert('RGB'), tensor=True).reshape(512)
        cos_sim = cos(vec1.unsqueeze(0),vec2.unsqueeze(0))
        if(cos_sim!=1): #no quiero devolver la propia imagen cono recomendación
            dic[im]=cos_sim
            similitudes.append(cos_sim)
    n_mayor_similitud=sorted(list(set(similitudes)), reverse=True)[:n]
    for im in imagenes:
        if(dic.get(im) in n_mayor_similitud):
            recomendadas[im]=dic.get(im)
            
    return recomendadas 

**Versión optimizada utilizando librería heapq para algoritmo de montículo**

Esta versión si que devuelve ya las recomendaciones ordenadas de mayor a menor similitud

In [ ]:
import heapq

def N_mas_parecidas_optimizada(imagen,n,imagenes):
    
    vec1 = img2vec.get_vec(imagen, tensor=True).reshape(512) #representación vectorial imagen de entrada
   
    heap=[] #lista de tuplas (similitud,imagen)

    for im in imagenes: 
        candidata = Image.open('../input/fashion-product-images-small/images/'+im)       
        vec2 = img2vec.get_vec(candidata.convert('RGB'), tensor=True).reshape(512)
        cos_sim = cos(vec1.unsqueeze(0),vec2.unsqueeze(0))
        if(cos_sim<1): #no quiero devolver la propia imagen cono recomendación
            
            heapq.heappush(heap, (round(cos_sim.item(),7), im))
    
    
    return heapq.nlargest(n, heap) 

Probamos las funciones

In [ ]:
dic=N_mas_parecidas(img,5,imagenes)

similares=[]
for nombre in dic.keys():
     similares.append(nombre)
print(dic)
print(similares)
rec1 = Image.open('../input/fashion-product-images-small/images/'+similares[0]) 
rec2 = Image.open('../input/fashion-product-images-small/images/'+similares[1]) 
rec3 = Image.open('../input/fashion-product-images-small/images/'+similares[2]) 
rec4 = Image.open('../input/fashion-product-images-small/images/'+similares[3]) 
rec5 = Image.open('../input/fashion-product-images-small/images/'+similares[4]) 
 

imagen clasificada:

In [ ]:
img  

recomendaciones:

In [ ]:
rec1

In [ ]:
rec2

In [ ]:
rec3

In [ ]:
rec4

In [ ]:
rec5

In [ ]:
lista=N_mas_parecidas_optimizada(img,5,imagenes)

similares=[]
for tupla in lista:
     similares.append(tupla[1])
print(lista)
print(similares)
rec1 = Image.open('../input/fashion-product-images-small/images/'+similares[0]) 
rec2 = Image.open('../input/fashion-product-images-small/images/'+similares[1]) 
rec3 = Image.open('../input/fashion-product-images-small/images/'+similares[2]) 
rec4 = Image.open('../input/fashion-product-images-small/images/'+similares[3]) 
rec5 = Image.open('../input/fashion-product-images-small/images/'+similares[4]) 

In [ ]:
img

In [ ]:
rec1

In [ ]:
rec2

In [ ]:
rec3

In [ ]:
rec4

In [ ]:
rec5

# COMPARACIÓN DEL RENDIMIENTO DE MI CNN CLASIFICADORA CON OTRAS PRE-ENTRENADAS


Para usar los modelos pre-entrenados para clasificar un dataset propio hay que re-entrenarlos con mi propio dataset. Para hacer esto, debo cambiar la última capa de clasificación para que tenga un número de salidas igual al número de clases en mi dataset. Luego, debo entrenar los modelos con mi dataset de forma que se ajusten a las características de mis imágenes y las clases que intento clasificar. Finalmente, puedo evaluar la precisión del modelo en mis datos de prueba.

In [ ]:
import torchvision.models as models


 La normalización para imagenes de ImageNet suele ser similar para muchos modelos preentrenados, incluyendo ResNet18, VGG16 y DenseNet121. La normalización típica consiste en restar la media de los valores de los canales RGB y dividir por la desviación estándar. Los valores concretos pueden variar, pero a menudo se usan [0.485, 0.456, 0.406] y [0.229, 0.224, 0.225] como valores de media y desviación estándar, respectivamente.
Creo un dataset que se normalizará usando las transformaciones esperadas por modelos preentrenados como ResNet, VGG y DenseNet. 
 

In [ ]:

transformaciones_imagenet = transforms.Compose([transforms.Resize((224,224)),
                                      transforms.ToTensor(), 
                                      transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) 
                                     ])
ImageNetTransformsdataset =CustomDataset(
    csv_file="./custom.csv",
    root_dir="../input/fashion-product-images-small/images/",
    transform=transformaciones_imagenet,
)
#Dividimos el dataset en 2:entrenamiento y test, y creamos los dataloaders
train_set_imageNet, test_set_imageNet = torch.utils.data.random_split(dataset, [30000, 14072])
train_loader_imageNet = DataLoader(dataset=train_set_imageNet, batch_size=batch_size, shuffle=True, pin_memory=True, num_workers=2)
test_loader_imageNet = DataLoader(dataset=test_set_imageNet, batch_size=batch_size, num_workers=2)





In [ ]:
print(len(train_set_imageNet),len(train_loader_imageNet))

# Fine Tuning con resnet-18

Para implementar las funciones de entrenamiento a mano, me he basado en esta guía: 

 [tutorial transfer learning](https://medium.com/nerd-for-tech/image-classification-using-transfer-learning-pytorch-resnet18-32b642148cbe)

In [ ]:

# Definir el modelo ResNet-18
model = models.resnet18(pretrained=True)

# Modificamos la salida del modelo para adaptarse a 25 clases
num_ftrs = model.fc.in_features  #obtenemos el numero de neuronas de entrada de la capa de salida en resnet
model.fc = nn.Linear(num_ftrs, 25) #sustituimos la capa final por una con 25 neuronas
model = model.to(device)


# Definir la función de pérdida y el optimizador (utilizo las mismas que en mi modelo)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)



In [ ]:

num_epochs = 11
for epoch in range(num_epochs): #(loop for every epoch)
    print("Epoch {} running".format(epoch)) #(printing message)
    """ Training Phase """
    model.train()    #(training model)
    running_loss = 0.   #(set loss 0)
    running_corrects = 0 
    # load a batch data of images
    for i, (inputs, labels) in enumerate(train_loader_imageNet):
        inputs = inputs.to(device)
        labels = labels.to(device) 
        # forward inputs and get output
        optimizer.zero_grad()
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)
        # get loss value and update the network weights
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
    epoch_loss = running_loss / len(train_set_imageNet)  #divido por el tamaño del dataset, ya que una época recorre todo el dataset
    epoch_acc = running_corrects / len(train_set_imageNet) * 100.
    print('[Train #{}] Loss: {:.4f} Acc: {:.4f}%'.format(epoch, epoch_loss, epoch_acc))

In [ ]:
torch.save(model.state_dict(), "resnet18.pth")

In [ ]:


resnet18_model = models.resnet18(pretrained=True)  
num_features = resnet18_model.fc.in_features 
resnet18_model.fc = nn.Linear(num_features, 25)
resnet18_model.load_state_dict(torch.load("resnet18.pth"))
resnet18_model.to(device)


In [ ]:
#PROBAMOS A PREDECIR 1 DE LAS IMAGENES DEL TEST_LOADER


batch=next(iter(test_loader_imageNet)) #cogemos 1 lote, batch=[[imgs],[labels]]
imgs, labels= batch[0],batch[1]
im=imgs[1]
label=labels[1]
plt.imshow(im.numpy()[0], cmap='gray')
print("valor esperado:",classDic[label.item()])

In [ ]:
output = resnet18_model(im.unsqueeze(0).to(device))
_, predicted = torch.max(output, 1)
etiqueta =classDic[predicted.item()]
print('Valor predicho: ',etiqueta )

In [ ]:

import time

##Testing

start_time = time.time()
criterion = nn.CrossEntropyLoss()


#Calculamos la acc y loss del modelo 
with torch.no_grad():
        correct = 0
        total = 0
        total_loss=0
        resnet18_model.eval()
        for data, target in test_loader_imageNet: #vamos iterando los distintos lotes del test_loader
            images = data
            labels =target
            images = images.to('cuda')
            labels = labels.to('cuda')
            outputs = resnet18_model(images)
            total_loss += criterion(outputs,labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
                

            accuracy = correct / total

        print('Test Accuracy of the model: {} %'.format(100 * correct / total))
        print('loss',total_loss/len(test_loader_imageNet))


# Fine Tuning con vgg16

In [ ]:
# Definir el modelo ResNet-18
model = models.vgg16(pretrained=True)
#vgg16 tiene 7 capas lineales
num_ftrs = model.classifier[6].in_features #obtenemos el numero de neuronas de entrada de la capa de salida en vgg
model.classifier[6] = nn.Linear(num_ftrs, 25) #sustituimos la capa final por una con 25 neuronas
model = model.to(device)

# Definir la función de pérdida y el optimizador (utilizo las mismas que en mi modelo)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)



In [ ]:

num_epochs = 10
for epoch in range(num_epochs): #(loop for every epoch)
    print("Epoch {} running".format(epoch)) #(printing message)
    """ Training Phase """
    model.train()    #(training model)
    running_loss = 0.   #(set loss 0)
    running_corrects = 0 
    # load a batch data of images
    for i, (inputs, labels) in enumerate(train_loader_imageNet):
        inputs = inputs.to(device)
        labels = labels.to(device) 
        # forward inputs and get output
        optimizer.zero_grad()
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)
        # get loss value and update the network weights
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
    epoch_loss = running_loss / len(train_set_imageNet)  #divido por el tamaño del dataset, ya que una época recorre todo el dataset
    epoch_acc = running_corrects / len(train_set_imageNet) * 100.
    print('[Train #{}] Loss: {:.4f} Acc: {:.4f}%'.format(epoch, epoch_loss, epoch_acc))

In [ ]:
torch.save(model.state_dict(), "vgg16.pth")

In [ ]:
vgg16_model = models.vgg16(pretrained=True)  
num_features = vgg16_model.classifier[6].in_features 
vgg16_model.classifier[6] = nn.Linear(num_features, 25)
vgg16_model.load_state_dict(torch.load("vgg16.pth"))
vgg16_model.to(device)



In [ ]:

import torchvision


##Testing

start_time = time.time()
criterion = nn.CrossEntropyLoss()


#Calculamos la acc y loss del modelo 
with torch.no_grad():
        correct = 0
        total = 0
        total_loss=0
        vgg16_model.eval()
        for data, target in test_loader_imageNet: #vamos iterando los distintos lotes del test_loader
            images = data
            labels =target
            images = images.to('cuda')
            labels = labels.to('cuda')
            outputs = vgg16_model(images)
            total_loss += criterion(outputs,labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
                

            accuracy = correct / total

        print('Test Accuracy of the model: {} %'.format(100 * correct / total))
        print('loss',total_loss/len(test_loader_imageNet))

# Fine Tuning con densenet121

In [ ]:
# Definir el modelo DenseNet-121
model = models.densenet121(pretrained=True)

# Modificamos la salida del modelo para adaptarse a 25 clases
num_ftrs = model.classifier.in_features #obtenemos el numero de neuronas de entrada de la capa de salida en DenseNet
model.classifier = nn.Linear(num_ftrs, 25) #sustituimos la capa final por una con 25 neuronas
model = model.to(device)
# Definir la función de pérdida y el optimizador (utilizo las mismas que en mi modelo)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

In [ ]:
num_epochs = 30
for epoch in range(num_epochs): #(loop for every epoch)
    print("Epoch {} running".format(epoch)) #(printing message)
    """ Training Phase """
    model.train()    #(training model)
    running_loss = 0.   #(set loss 0)
    running_corrects = 0 
    # load a batch data of images
    for i, (inputs, labels) in enumerate(train_loader_imageNet):
        inputs = inputs.to(device)
        labels = labels.to(device) 
        # forward inputs and get output
        optimizer.zero_grad()
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)
        # get loss value and update the network weights
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
    epoch_loss = running_loss / len(train_set_imageNet)  #divido por el tamaño del dataset, ya que una época recorre todo el dataset
    epoch_acc = running_corrects / len(train_set_imageNet) * 100.
    print('[Train #{}] Loss: {:.4f} Acc: {:.4f}%'.format(epoch, epoch_loss, epoch_acc))

In [ ]:
torch.save(model.state_dict(), "densenet121.pth")

In [ ]:

densenet121_model = models.densenet121(pretrained=True)
num_ftrs = densenet121_model.classifier.in_features 
densenet121_model.classifier = nn.Linear(num_ftrs, 25) 

densenet121_model.load_state_dict(torch.load("densenet121.pth"))
densenet121_model = densenet121_model.to(device)

In [ ]:

#Calculamos la acc y loss del modelo 
with torch.no_grad():
        correct = 0
        total = 0
        total_loss=0
        densenet121_model.eval()
        for data, target in test_loader_imageNet: #vamos iterando los distintos lotes del test_loader
            images = data
            labels =target
            images = images.to('cuda')
            labels = labels.to('cuda')
            outputs = densenet121_model(images)
            total_loss += criterion(outputs,labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
                

            accuracy = correct / total

        print('Test Accuracy of the model: {} %'.format(100 * correct / total))
        print('loss',total_loss/len(test_loader_imageNet))

# PRUEBAS FINE-TUNING CONGELANDO LAS PRIMERAS CAPAS
# 

# Resnet18

Pruebo a congelar todas las secuencias de capas convolucionales excepto la última (layer4). Por lo que solo actualizaría los pesos de las capas de la última secuencia de convoluciones y de la capa de clasificación. Las capas convolucionales en los modelos de redes neuronales suelen ser agrupadas en secuencias lógicas, y en ResNet18 estas secuencias se nombran como "layer1", "layer2", "layer3" y "layer4". Cada una de estas secuencias tiene varias capas convolucionales dentro de ellas.

In [ ]:

# Definir el modelo ResNet-18
model = models.resnet18(pretrained=True)

# Congelar las primeras 15 capas convolucionales
for i, param in model.named_parameters():
    if 'layer4' not in i and 'fc' not in i:
        param.requires_grad = False
        
# Modificamos la salida del modelo para adaptarse a 25 clases
num_ftrs = model.fc.in_features  #obtenemos el numero de neuronas de entrada de la capa de salida en resnet
model.fc = nn.Linear(num_ftrs, 25) #sustituimos la capa final por una con 25 neuronas
model = model.to(device)


# Definir la función de pérdida y el optimizador (utilizo las mismas que en mi modelo)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

In [ ]:

num_epochs = 20
for epoch in range(num_epochs): #(loop for every epoch)
    print("Epoch {} running".format(epoch)) #(printing message)
    """ Training Phase """
    model.train()    #(training model)
    running_loss = 0.   #(set loss 0)
    running_corrects = 0 
    # load a batch data of images
    for i, (inputs, labels) in enumerate(train_loader_imageNet):
        inputs = inputs.to(device)
        labels = labels.to(device) 
        # forward inputs and get output
        optimizer.zero_grad()
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)
        # get loss value and update the network weights
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
    epoch_loss = running_loss / len(train_set_imageNet)  #divido por el tamaño del dataset, ya que una época recorre todo el dataset
    epoch_acc = running_corrects / len(train_set_imageNet) * 100.
    print('[Train #{}] Loss: {:.4f} Acc: {:.4f}%'.format(epoch, epoch_loss, epoch_acc))

In [ ]:
torch.save(model.state_dict(), "resnet18-2.pth")

In [ ]:


resnet18_model = models.resnet18(pretrained=True)  

# Congelar las primeras 15 capas convolucionales
for i, param in resnet18_model.named_parameters():
    if 'layer4' not in i and 'fc' not in i:
        param.requires_grad = False
        
num_features = resnet18_model.fc.in_features 
resnet18_model.fc = nn.Linear(num_features, 25)
resnet18_model.load_state_dict(torch.load("resnet18-2.pth"))
resnet18_model.to(device)

In [ ]:

import time

##Testing

start_time = time.time()
criterion = nn.CrossEntropyLoss()


#Calculamos la acc y loss del modelo 
with torch.no_grad():
        correct = 0
        total = 0
        total_loss=0
        resnet18_model.eval()
        for data, target in test_loader_imageNet: #vamos iterando los distintos lotes del test_loader
            images = data
            labels =target
            images = images.to('cuda')
            labels = labels.to('cuda')
            outputs = resnet18_model(images)
            total_loss += criterion(outputs,labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
                

            accuracy = correct / total

        print('Test Accuracy of the model: {} %'.format(100 * correct / total))
        print('loss',total_loss/len(test_loader_imageNet))


# **Análisis de las etiquetas del dataset**




A continuación se analiza la representación en el dataset de las distintas clases obtenidas al combinar la etiqueta de género con el resto del dataset, para intentar solucionar el segundo problema explicado en la memoria. El objetivo es el de obtener el porcentaje de muestras del dataset obtenida para cada clase en cada combinación de etiquetas.


# Representación de las etiquetas

In [ ]:
usage= dataFrame.usage.unique()

porcentajesUsage={}
for etiqueta in usage:
    perc = (((dataFrame['usage'] == etiqueta)).sum() / dataFrame.shape[0]) * 100
    porcentajesUsage[etiqueta] = round(perc,4)
print(porcentajesUsage)

In [ ]:
masterCategory= dataFrame.masterCategory.unique()

porcentajesMasterCategory={}
for etiqueta in masterCategory:
    perc = (((dataFrame['masterCategory'] == etiqueta)).sum() / dataFrame.shape[0]) * 100
    porcentajesMasterCategory[etiqueta] = round(perc,4)
print(porcentajesMasterCategory)

In [ ]:
subCategory= dataFrame.subCategory.unique()

porcentajesSubCategory={}
for etiqueta in subCategory:
    perc = (((dataFrame['subCategory'] == etiqueta)).sum() / dataFrame.shape[0]) * 100
    porcentajesSubCategory[etiqueta] = round(perc,4)
print(porcentajesSubCategory)

In [ ]:
articleType= dataFrame.articleType.unique()

porcentajesArticleType={}
for etiqueta in articleType:
    perc = (((dataFrame['articleType'] == etiqueta)).sum() / dataFrame.shape[0]) * 100
    porcentajesArticleType[etiqueta] = round(perc,4)
print(porcentajesArticleType)

# Representación de las clases fruto de combinar las distintas etiquetas con la etiqueta género
# 

**Usage**

In [ ]:
#creo la columna image con id+.jpg y la columna target con masterCategory+gender

def getImageName(id): return '{name}.jpg'.format(name = id)

dataFrame['image'] = dataFrame['id'].map(getImageName)
dataFrame['target'] =dataFrame['usage']+dataFrame['gender']

dataFrame.head()


In [ ]:
#nos quedamos solo con las columnas image y target
ds = dataFrame.filter(['image', 'target'])
ds.head()


In [ ]:
targets=ds.target.unique()
CLASSES=len(targets)
print(CLASSES)
print(targets)

In [ ]:
#calculamos la representacion que se obtiene para cada valor
porcentajes={}
for target in targets:
    perc = (((dataFrame['target'] == target)).sum() / dataFrame.shape[0]) * 100
    porcentajes[target] = round(perc,4)
print(porcentajes)

**Master Category**

In [ ]:

dataFrame['image'] = dataFrame['id'].map(getImageName)
dataFrame['target'] =dataFrame['masterCategory']+dataFrame['gender']
ds = dataFrame.filter(['image', 'target'])
targets=ds.target.unique()
CLASSES=len(targets)
porcentajes={}
for target in targets:
    perc = (((dataFrame['target'] == target)).sum() / dataFrame.shape[0]) * 100
    porcentajes[target] = round(perc,4)
print(porcentajes)

**subCategory**

In [ ]:

dataFrame['image'] = dataFrame['id'].map(getImageName)
dataFrame['target'] =dataFrame['subCategory']+dataFrame['gender']
ds = dataFrame.filter(['image', 'target'])
targets=ds.target.unique()
CLASSES=len(targets)
porcentajes={}
for target in targets:
    perc = (((dataFrame['target'] == target)).sum() / dataFrame.shape[0]) * 100
    porcentajes[target] = round(perc,4)
print(porcentajes)

**articleType**

In [ ]:
dataFrame['image'] = dataFrame['id'].map(getImageName)
dataFrame['target'] =dataFrame['articleType']+dataFrame['gender']
ds = dataFrame.filter(['image', 'target'])
targets=ds.target.unique()
CLASSES=len(targets)
porcentajes={}
for target in targets:
    perc = (((dataFrame['target'] == target)).sum() / dataFrame.shape[0]) * 100
    porcentajes[target] = round(perc,4)
print(porcentajes)

La única para la que se podrían obtener mejores resultados que con usage, es para la etiqueta mastercategory, por lo que copiaré este notebook y volveré a entrenar el sistema usando dicha etiqueta para comprobar si mejora el rendimiento.

UPDATE: No hay ninguna mejora significativa tras el cambio de etiqueta, por lo que este es el notebook final.